In [4]:
#!pip install selenium

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
#Imports:
import selenium
from selenium import webdriver
import time
import pandas as pd
import re
import tqdm
import datetime
import dateutil.parser as dparser
from datetime import timedelta

In [2]:
#Constants:
URL_TO_SCRAPE = "https://www.instagram.com/aleksandrskrivickis"
    #Scraper delays(s):
DELAY_GETPOSTDATA = 0.1
DELAY_GETALLPOSTDATA = 0.1
DELAY_SCROLLER = 1
DELAY_COMMENT_EXPANDER = 0.1

In [3]:
def scrollPageToBottomAndFindPostLinks():
    allPosts = []
    
    prevHeight = 0
    newHeight = 1

    while prevHeight != newHeight:
        prevHeight = int(driver.execute_script("return document.body.scrollHeight;"))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(DELAY_SCROLLER)
        allPosts.append(findPostLinks(driver))
        newHeight = int(driver.execute_script("return document.body.scrollHeight;"))
        
    return allPosts; 

def findPostLinks(driver):
    posts = []
    src = driver.page_source
    src_splitted = src.split("</div>")

    for entry in src_splitted:
        postRegex = re.findall("(<a href=\")(.*)(\?taken-by=\w*\">)", entry)
        if len(postRegex) > 0:
            posts.append(postRegex[0][1])
    return posts

def getPostData(driver):
    time.sleep(DELAY_GETPOSTDATA)
    postData = []
    expandAllComments(driver)
    article = driver.find_element_by_tag_name("article")
    post_link = driver.current_url
    image_link = driver.find_elements_by_tag_name('img')[1].get_attribute('src')
    author = article.text.splitlines()[0]
    authors, comments = getAllCommentsFromArticle(driver)
    #Getting likes
    try:
        likes = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/section[2]/div/span/span").text#driver.find_element_by_tag_name('span').text.splitlines()[6].replace(" likes", "")
    except Exception as e:
        print("Unable to get likes from post: " + post_link + "\n Trying different approach...")
        try:
            likes = (len(driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/section[2]/div").text.split(",")) + 1)
        except Exception as Ex:
            print("Different approach didn't work. Value of likes field will be \"Exception\"")
            likes = "Exception"
            
    #Getting and processing date
    try:
        date = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/div[2]/a/time").text#driver.find_element_by_tag_name('article').text.splitlines()[len(driver.find_element_by_tag_name('article').text.splitlines()) - 2]
    except Exception as e:
        print("Unable to get date from post: " + post_link)
        date = "Exception"
        
    if ("," not in date):
        date = date + ", " + str(datetime.datetime.now().year)
    date = convertDate(date)
    
    if date == "" or " days ago" in driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/div[2]/a/time").text.casefold():
        daysAgo = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/div[2]/a/time").text.casefold().replace(" days ago", "")
        date = datetime.datetime.now() - timedelta(days=int(daysAgo))
        

    firstRun = True
    for a, c in zip(authors, comments):
        if firstRun:
            firstRun = False
            postData.append({"post_link" : post_link, "image_link" : image_link, "post_author" : a, "post" : c, "likes": likes, "date": date, "comment": "", "comment_author": ""})
        else:
            postData.append({"post_link" : post_link, "image_link" : image_link, "post_author" : "N/A", "post" : "N/A", "likes": "N/A", "date": date, "comment": c, "comment_author": a})
    return postData

def getAllCommentsFromArticle(driver):
    #Posts description and authors name is a very first comment's content and authors name
    authors = []
    comments = []
    article = driver.find_element_by_tag_name("article")
    comment = article.find_elements_by_tag_name("li")
    for com in comment:
        #print("\n" + com.find_element_by_tag_name("a").text + ", post: " + com.find_element_by_tag_name("span").text)
        authors.append(com.find_element_by_tag_name("a").text)
        comments.append(com.find_element_by_tag_name("span").text)
    return authors, comments

def expandAllComments(driver):
    try:
        el = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/div[1]/ul/li[2]/a")
        while (el.text == "Load more comments" or "View all" in el.text):
            el = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/div[1]/ul/li[2]/a")
            if (el.text == "Load more comments" or "View all" in el.text):
                el.click()
            time.sleep(DELAY_COMMENT_EXPANDER)
    except Exception as e:
        print("expandAllComments() - expanded")
        
def convertDate(date):
    returnable = []
    try:
        if (str(date) == "NaT" or str(date) == ""):
            currDate = str(date)
            returnable == ""
        else:
            currDate = str(date)
            returnable = dparser.parse(str(date)).date()
    except Exception as e:
        print("Exception in convertAllDates() " + str(e))
        print("Exception in convertAllDates() caused by " + currDate + " instead of date string...")
        returnable = ""
    return returnable           

In [4]:
if __name__ == "__main__":
    
    #Getting all the post links
    driver = webdriver.Chrome()
    driver.get(URL_TO_SCRAPE)
    allPosts = scrollPageToBottomAndFindPostLinks()
    
    #Concatenate all lists in one
    allPosts = sum(allPosts, [])
    #Remove dupes from posts
    allPosts = list(set(allPosts))
    allPosts = ["https://www.instagram.com" + x for x in allPosts]
    print("Amount of links to posts scraped: " + str(len(allPosts)))
    allPostData = []
    
    #Parsing every post
    #driver = webdriver.Chrome()
    for link in tqdm.tqdm(allPosts):
        driver.get(link)
        time.sleep(DELAY_GETALLPOSTDATA)
        allPostData.append(getPostData(driver))
        
    #Export to file:
    allPostData = sum(allPostData, [])
    df = pd.DataFrame.from_dict(allPostData)
    df = df.reindex(['post_link', "image_link", "post", "date", "post_author", "likes", "comment", "comment_author"], axis=1)
    df.index.name = "entry"
    #Sort by post date, keep post and comment order
    df.reset_index(level=0, inplace=True)
    df['date'] = pd.to_datetime(df.date)
    df_sorted = df.sort_values(['date', 'entry'], ascending=[True, True])
    df_sorted.reset_index(inplace=True)
    df_sorted = df_sorted.iloc[::-1]
    df_sorted.drop(labels=["index", "entry"], axis=1, inplace=True)
    df_sorted.to_excel("aleksandrskrivickis_allposts_current_sorted_reversed.xlsx", index=False)

    driver.quit()

Amount of links to posts scraped: 127


  1%|▋                                                                                 | 1/127 [00:01<03:09,  1.51s/it]

expandAllComments() - expanded


  2%|█▎                                                                                | 2/127 [00:02<02:45,  1.32s/it]

expandAllComments() - expanded


...

 70%|████████████████████████████████████████████████████████▊                        | 89/127 [01:43<00:44,  1.16s/it]

expandAllComments() - expanded


 72%|██████████████████████████████████████████████████████████                       | 91/127 [01:45<00:41,  1.16s/it]

expandAllComments() - expanded
Exception in convertAllDates() Unknown string format
Exception in convertAllDates() caused by 1 DAY AGO, 2018 instead of date string...


ValueError: invalid literal for int() with base 10: '1 day ago'